<a href="https://colab.research.google.com/github/Suryash-Malviya/TTS/blob/main/suryash/StyleTTS2_Demo_LJSpeech_(3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install packages and download models

In [53]:
# This is a new part for testing

In [54]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [55]:
%%shell
# Setup the project directory and ensure that we do not repeat cloning or installations if they already exist
mkdir -p /content/drive/MyDrive/StyleTTS2
cd /content/drive/MyDrive/StyleTTS2

# Clone the repository if it doesn't already exist and setup the environment
if [ ! -d "StyleTTS2" ]; then
    git clone https://github.com/yl4579/StyleTTS2.git
    cd StyleTTS2
else
    cd StyleTTS2
fi

# Check if dependencies are already installed, install if not
pip freeze | grep SoundFile || pip install SoundFile torchaudio munch torch pydub pyyaml librosa nltk matplotlib accelerate transformers phonemizer datasets einops einops-exts tqdm typing-extensions git+https://github.com/resemble-ai/monotonic_align.git
dpkg -l | grep espeak-ng || sudo apt-get install espeak-ng

# Download models using git-lfs if they aren't already downloaded
if [ ! -d "Models" ]; then
    git-lfs clone https://huggingface.co/yl4579/StyleTTS2-LJSpeech
    mv StyleTTS2-LJSpeech/Models .
fi


  Cloning https://github.com/resemble-ai/monotonic_align.git to /tmp/pip-req-build-tghd1h6e
  Running command git clone --filter=blob:none --quiet https://github.com/resemble-ai/monotonic_align.git /tmp/pip-req-build-tghd1h6e
  Resolved https://github.com/resemble-ai/monotonic_align.git to commit 78b985be210a03d08bc3acc01c4df0442105366f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ii  espeak-ng                              1.50+dfsg-10ubuntu0.1                   amd64        Multi-lingual software speech synthesizer
ii  espeak-ng-data:amd64                   1.50+dfsg-10ubuntu0.1                   amd64        Multi-lingual software speech synthesizer: speech data files
ii  libespeak-ng1:amd64                    1.50+dfsg-10ubuntu0.1                   amd64        Multi-lingual software speech synthesizer: shared library


In [81]:
%%shell
git clone https://github.com/yl4579/StyleTTS2.git
cd StyleTTS2
pip install SoundFile torchaudio munch torch pydub pyyaml librosa nltk matplotlib accelerate transformers phonemizer datasets einops einops-exts tqdm typing-extensions git+https://github.com/resemble-ai/monotonic_align.git
sudo apt-get install espeak-ng
git-lfs clone https://huggingface.co/yl4579/StyleTTS2-LJSpeech
mv StyleTTS2-LJSpeech/Models .

Cloning into 'StyleTTS2'...
remote: Enumerating objects: 372, done.
remote: Total 372 (delta 0), reused 0 (delta 0), pack-reused 372 (from 1)
Receiving objects: 100% (372/372), 133.98 MiB | 24.16 MiB/s, done.
Resolving deltas: 100% (199/199), done.
Updating files: 100% (48/48), done.
  Cloning https://github.com/resemble-ai/monotonic_align.git to /tmp/pip-req-build-zy2125mf
  Running command git clone --filter=blob:none --quiet https://github.com/resemble-ai/monotonic_align.git /tmp/pip-req-build-zy2125mf
  Resolved https://github.com/resemble-ai/monotonic_align.git to commit 78b985be210a03d08bc3acc01c4df0442105366f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
espeak-ng is already the newest version (1.50+dfsg-10ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
          with 

### Load models

In [57]:
!pip install datasets transformers
%pip install munch


In [58]:
%pwd

'/content/StyleTTS2'

In [59]:
%cd StyleTTS2

import torch
torch.manual_seed(0)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

import random
random.seed(0)

import numpy as np
np.random.seed(0)

import nltk
nltk.download('punkt_tab')

# load packages
import time
import random
import yaml
from munch import Munch
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import torchaudio
import librosa
from nltk.tokenize import word_tokenize

from models import *
from utils import *
from text_utils import TextCleaner
textclenaer = TextCleaner()

[Errno 2] No such file or directory: 'StyleTTS2'
/content/StyleTTS2
177


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [60]:

%matplotlib inline

device = 'cuda' if torch.cuda.is_available() else 'cpu'

to_mel = torchaudio.transforms.MelSpectrogram(
    n_mels=80, n_fft=2048, win_length=1200, hop_length=300)
mean, std = -4, 4

In [61]:

def length_to_mask(lengths):
    mask = torch.arange(lengths.max()).unsqueeze(0).expand(lengths.shape[0], -1).type_as(lengths)
    mask = torch.gt(mask+1, lengths.unsqueeze(1))
    return mask

def preprocess(wave):
    wave_tensor = torch.from_numpy(wave).float()
    mel_tensor = to_mel(wave_tensor)
    mel_tensor = (torch.log(1e-5 + mel_tensor.unsqueeze(0)) - mean) / std
    return mel_tensor

def compute_style(ref_dicts):
    reference_embeddings = {}
    for key, path in ref_dicts.items():
        wave, sr = librosa.load(path, sr=24000)
        audio, index = librosa.effects.trim(wave, top_db=30)
        if sr != 24000:
            audio = librosa.resample(audio, sr, 24000)
        mel_tensor = preprocess(audio).to(device)

        with torch.no_grad():
            ref = model.style_encoder(mel_tensor.unsqueeze(1))
        reference_embeddings[key] = (ref.squeeze(1), audio)

    return reference_embeddings

# load phonemizer
import phonemizer
global_phonemizer = phonemizer.backend.EspeakBackend(language='en-us', preserve_punctuation=True, with_stress=True, words_mismatch='ignore')

config = yaml.safe_load(open("Models/LJSpeech/config.yml"))

# load pretrained ASR model
ASR_config = config.get('ASR_config', False)
ASR_path = config.get('ASR_path', False)
text_aligner = load_ASR_models(ASR_path, ASR_config)

# load pretrained F0 model
F0_path = config.get('F0_path', False)
pitch_extractor = load_F0_models(F0_path)

# load BERT model
from Utils.PLBERT.util import load_plbert
BERT_path = config.get('PLBERT_dir', False)
plbert = load_plbert(BERT_path)

model = build_model(recursive_munch(config['model_params']), text_aligner, pitch_extractor, plbert)
_ = [model[key].eval() for key in model]
_ = [model[key].to(device) for key in model]

params_whole = torch.load("Models/LJSpeech/epoch_2nd_00100.pth", map_location='cpu')
params = params_whole['net']

for key in model:
    if key in params:
        print('%s loaded' % key)
        try:
            model[key].load_state_dict(params[key])
        except:
            from collections import OrderedDict
            state_dict = params[key]
            new_state_dict = OrderedDict()
            for k, v in state_dict.items():
                name = k[7:] # remove `module.`
                new_state_dict[name] = v
            # load params
            model[key].load_state_dict(new_state_dict, strict=False)
#             except:
#                 _load(params[key], model[key])
_ = [model[key].eval() for key in model]

from Modules.diffusion.sampler import DiffusionSampler, ADPM2Sampler, KarrasSchedule

sampler = DiffusionSampler(
    model.diffusion.diffusion,
    sampler=ADPM2Sampler(),
    sigma_schedule=KarrasSchedule(sigma_min=0.0001, sigma_max=3.0, rho=9.0), # empirical parameters
    clamp=False
)

def inference(text, noise, diffusion_steps=5, embedding_scale=1):
    text = text.strip()
    text = text.replace('"', '')
    ps = global_phonemizer.phonemize([text])
    ps = word_tokenize(ps[0])
    ps = ' '.join(ps)

    tokens = textclenaer(ps)
    tokens.insert(0, 0)
    tokens = torch.LongTensor(tokens).to(device).unsqueeze(0)

    with torch.no_grad():
        input_lengths = torch.LongTensor([tokens.shape[-1]]).to(tokens.device)
        text_mask = length_to_mask(input_lengths).to(tokens.device)

        t_en = model.text_encoder(tokens, input_lengths, text_mask)
        bert_dur = model.bert(tokens, attention_mask=(~text_mask).int())
        d_en = model.bert_encoder(bert_dur).transpose(-1, -2)

        s_pred = sampler(noise,
              embedding=bert_dur[0].unsqueeze(0), num_steps=diffusion_steps,
              embedding_scale=embedding_scale).squeeze(0)

        s = s_pred[:, 128:]
        ref = s_pred[:, :128]

        d = model.predictor.text_encoder(d_en, s, input_lengths, text_mask)

        x, _ = model.predictor.lstm(d)
        duration = model.predictor.duration_proj(x)
        duration = torch.sigmoid(duration).sum(axis=-1)
        pred_dur = torch.round(duration.squeeze()).clamp(min=1)

        pred_dur[-1] += 5

        pred_aln_trg = torch.zeros(input_lengths, int(pred_dur.sum().data))
        c_frame = 0
        for i in range(pred_aln_trg.size(0)):
            pred_aln_trg[i, c_frame:c_frame + int(pred_dur[i].data)] = 1
            c_frame += int(pred_dur[i].data)

        # encode prosody
        en = (d.transpose(-1, -2) @ pred_aln_trg.unsqueeze(0).to(device))
        F0_pred, N_pred = model.predictor.F0Ntrain(en, s)
        out = model.decoder((t_en @ pred_aln_trg.unsqueeze(0).to(device)),
                                F0_pred, N_pred, ref.squeeze().unsqueeze(0))

    return out.squeeze().cpu().numpy()

def LFinference(text, s_prev, noise, alpha=0.7, diffusion_steps=5, embedding_scale=1):
  text = text.strip()
  text = text.replace('"', '')
  ps = global_phonemizer.phonemize([text])
  ps = word_tokenize(ps[0])
  ps = ' '.join(ps)

  tokens = textclenaer(ps)
  tokens.insert(0, 0)
  tokens = torch.LongTensor(tokens).to(device).unsqueeze(0)

  with torch.no_grad():
      input_lengths = torch.LongTensor([tokens.shape[-1]]).to(tokens.device)
      text_mask = length_to_mask(input_lengths).to(tokens.device)

      t_en = model.text_encoder(tokens, input_lengths, text_mask)
      bert_dur = model.bert(tokens, attention_mask=(~text_mask).int())
      d_en = model.bert_encoder(bert_dur).transpose(-1, -2)

      s_pred = sampler(noise,
            embedding=bert_dur[0].unsqueeze(0), num_steps=diffusion_steps,
            embedding_scale=embedding_scale).squeeze(0)

      if s_prev is not None:
          # convex combination of previous and current style
          s_pred = alpha * s_prev + (1 - alpha) * s_pred

      s = s_pred[:, 128:]
      ref = s_pred[:, :128]

      d = model.predictor.text_encoder(d_en, s, input_lengths, text_mask)

      x, _ = model.predictor.lstm(d)
      duration = model.predictor.duration_proj(x)
      duration = torch.sigmoid(duration).sum(axis=-1)
      pred_dur = torch.round(duration.squeeze()).clamp(min=1)

      pred_aln_trg = torch.zeros(input_lengths, int(pred_dur.sum().data))
      c_frame = 0
      for i in range(pred_aln_trg.size(0)):
          pred_aln_trg[i, c_frame:c_frame + int(pred_dur[i].data)] = 1
          c_frame += int(pred_dur[i].data)

      # encode prosody
      en = (d.transpose(-1, -2) @ pred_aln_trg.unsqueeze(0).to(device))
      F0_pred, N_pred = model.predictor.F0Ntrain(en,
      s)
      out = model.decoder((t_en @ pred_aln_trg.unsqueeze(0).to(device)),
                              F0_pred, N_pred, ref.squeeze().unsqueeze(0))

  return out.squeeze().cpu().numpy(), s_pred

/content/StyleTTS2/models.py:604: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  params = torch.load(model_path, map_location='cpu')['model']
/content/StyleTTS2/models.py:588

bert loaded
bert_encoder loaded
predictor loaded
decoder loaded
text_encoder loaded
predictor_encoder loaded
style_encoder loaded
diffusion loaded
text_aligner loaded
pitch_extractor loaded
mpd loaded
msd loaded
wd loaded


In [62]:
def inference_accent(text, noise, diffusion_steps=5, embedding_scale=1, accent=None):
    # Process the text
    text = text.strip()
    text = text.replace('"', '')

    # Phonemize the text (considering accent)
    ps = global_phonemizer.phonemize([text], accent=accent)  # Adjust phonemization based on accent
    ps = word_tokenize(ps[0])
    ps = ' '.join(ps)

    tokens = textclenaer(ps)
    tokens.insert(0, 0)
    tokens = torch.LongTensor(tokens).to(device).unsqueeze(0)

    # Use accent to adjust embedding scale or other parameters (if necessary)
    if accent:
        accent_map = {
            'german': 0.8,  # Example scale factor for german accent
            'english': 1.2,  # Example scale factor for english accent
            # Add other accents and their respective adjustments here
        }
        embedding_scale *= accent_map.get(accent, 1)  # Adjust embedding scale based on accent

    with torch.no_grad():
        input_lengths = torch.LongTensor([tokens.shape[-1]]).to(tokens.device)
        text_mask = length_to_mask(input_lengths).to(tokens.device)

        t_en = model.text_encoder(tokens, input_lengths, text_mask)
        bert_dur = model.bert(tokens, attention_mask=(~text_mask).int())
        d_en = model.bert_encoder(bert_dur).transpose(-1, -2)

        s_pred = sampler(noise,
                          embedding=bert_dur[0].unsqueeze(0), num_steps=diffusion_steps,
                          embedding_scale=embedding_scale).squeeze(0)

        s = s_pred[:, 128:]
        ref = s_pred[:, :128]

        d = model.predictor.text_encoder(d_en, s, input_lengths, text_mask)

        x, _ = model.predictor.lstm(d)
        duration = model.predictor.duration_proj(x)
        duration = torch.sigmoid(duration).sum(axis=-1)
        pred_dur = torch.round(duration.squeeze()).clamp(min=1)

        pred_dur[-1] += 5

        pred_aln_trg = torch.zeros(input_lengths, int(pred_dur.sum().data))
        c_frame = 0
        for i in range(pred_aln_trg.size(0)):
            pred_aln_trg[i, c_frame:c_frame + int(pred_dur[i].data)] = 1
            c_frame += int(pred_dur[i].data)

        # encode prosody
        en = (d.transpose(-1, -2) @ pred_aln_trg.unsqueeze(0).to(device))
        F0_pred, N_pred = model.predictor.F0Ntrain(en, s)
        out = model.decoder((t_en @ pred_aln_trg.unsqueeze(0).to(device)),
                            F0_pred, N_pred, ref.squeeze().unsqueeze(0))

    return out.squeeze().cpu().numpy()


In [63]:
!pip install nltk
import nltk
nltk.download('punkt_tab') # download the 'punkt_tab' data package for sentence tokenization

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

### Synthesize speech

In [64]:
# @title Input Text { display-mode: "form" }
# synthesize a text
text = "we needed to install nltk!" # @param {type:"string"}


#### Basic synthesis (5 diffusion steps)

In [65]:
!grep -r "def inference" .


./models.py:    def inference(self, x):
./models.py:    def inference(self, x, style):
./Demo/Inference_LJSpeech.ipynb:    "def inference(text, noise, diffusion_steps=5, embedding_scale=1):\n",
./Demo/Inference_LibriTTS.ipynb:    "def inference(text, ref_s, alpha = 0.3, beta = 0.7, diffusion_steps=5, embedding_scale=1):\n",
./Colab/StyleTTS2_Demo_LibriTTS.ipynb:        "def inference(text, ref_s, alpha = 0.3, beta = 0.7, diffusion_steps=5, embedding_scale=1):\n",
./Colab/StyleTTS2_Demo_LJSpeech.ipynb:        "def inference(text, noise, diffusion_steps=5, embedding_scale=1):\n",
./Colab/StyleTTS2_Finetune_Demo.ipynb:        "def inference(text, ref_s, alpha = 0.3, beta = 0.7, diffusion_steps=5, embedding_scale=1):\n",


In [66]:
import time

start = time.time()
noise = torch.randn(1,1,256).to(device)
wav = inference(text, noise, diffusion_steps=5, embedding_scale=1)
rtf = (time.time() - start) / (len(wav) / 24000)
print(f"RTF = {rtf:5f}")
import IPython.display as ipd
display(ipd.Audio(wav, rate=24000))

RTF = 4.896541


#### With higher diffusion steps (more diverse)
Since the sampler is ancestral, the higher the stpes, the more diverse the samples are, with the cost of slower synthesis speed.

In [67]:
# start = time.time()
# noise = torch.randn(1,1,256).to(device)
# wav = inference(text, noise, diffusion_steps=10, embedding_scale=1)
# rtf = (time.time() - start) / (len(wav) / 24000)
# print(f"RTF = {rtf:5f}")
# import IPython.display as ipd
# display(ipd.Audio(wav, rate=24000))

### Speech expressiveness
The following section recreates the samples shown in [Section 6](https://styletts2.github.io/#emo) of the demo page.

#### With embedding_scale=1
This is the classifier-free guidance scale. The higher the scale, the more conditional the style is to the input text and hence more emotional.

In [68]:
# texts = {}
# texts['Happy'] = "We are happy to invite you to join us on a journey to the past, where we will visit the most amazing monuments ever built by human hands."
# texts['Sad'] = "I am sorry to say that we have suffered a severe setback in our efforts to restore prosperity and confidence."
# texts['Angry'] = "The field of astronomy is a joke! Its theories are based on flawed observations and biased interpretations!"
# texts['Surprised'] = "I can't believe it! You mean to tell me that you have discovered a new species of bacteria in this pond?"

# for k,v in texts.items():
#     noise = torch.randn(1,1,256).to(device)
#     wav = inference(v, noise, diffusion_steps=10, embedding_scale=1)
#     print(k + ": ")
#     display(ipd.Audio(wav, rate=24000, normalize=False))

#### With embedding_scale=2

In [69]:
# texts = {}
# texts['Happy'] = "We are happy to invite you to join us on a journey to the past, where we will visit the most amazing monuments ever built by human hands."
# texts['Sad'] = "I am sorry to say that we have suffered a severe setback in our efforts to restore prosperity and confidence."
# texts['Angry'] = "The field of astronomy is a joke! Its theories are based on flawed observations and biased interpretations!"
# texts['Surprised'] = "I can't believe it! You mean to tell me that you have discovered a new species of bacteria in this pond?"

# for k,v in texts.items():
#     noise = torch.randn(1,1,256).to(device)
#     wav = inference(v, noise, diffusion_steps=10, embedding_scale=2) # embedding_scale=2 for more pronounced emotion
#     print(k + ": ")
#     display(ipd.Audio(wav, rate=24000, normalize=False))

### Long-form generation
This section includes basic implementation of Algorithm 1 in the paper for consistent longform audio generation. The example passage is taken from [Section 5](https://styletts2.github.io/#long) of the demo page.

In [70]:
passage = '''If the supply of fruit is greater than the family needs, it may be made a source of income by sending the fresh fruit to the market if there is one near enough, or by preserving, canning, and making jelly for sale. To make such an enterprise a success the fruit and work must be first class. There is magic in the word "Homemade," when the product appeals to the eye and the palate; but many careless and incompetent people have found to their sorrow that this word has not magic enough to float inferior goods on the market. As a rule large canning and preserving establishments are clean and have the best appliances, and they employ chemists and skilled labor. The home product must be very good to compete with the attractive goods that are sent out from such establishments. Yet for first-class homemade products there is a market in all large cities. All first-class grocers have customers who purchase such goods.''' # @param {type:"string"}

In [71]:
# sentences = passage.split('.') # simple split by comma
# wavs = []
# s_prev = None
# for text in sentences:
#     if text.strip() == "": continue
#     text += '.' # add it back
#     noise = torch.randn(1,1,256).to(device)
#     wav, s_prev = LFinference(text, s_prev, noise, alpha=0.7, diffusion_steps=10, embedding_scale=1.5)
#     wavs.append(wav)
# display(ipd.Audio(np.concatenate(wavs), rate=24000, normalize=False))

### Streaming Data

In [72]:
!pip install datasets transformers

In [73]:
# Import all necessary libraries
from datasets import load_dataset, Dataset, DatasetDict
from transformers import AutoTokenizer, TrainingArguments, Trainer, DataCollatorWithPadding
import pandas as pd
import torch

# Load the tokenizer for the German BERT model
tokenizer = AutoTokenizer.from_pretrained('bert-base-german-dbmdz-uncased')


In [74]:
# from datasets import load_dataset

# # Load a dataset in streaming mode
# dataset = load_dataset("facebook/multilingual_librispeech", "german", split="train", streaming=True)



In [75]:
# # # def preprocess_data(batch):
# # #     # Tokenize the transcript
# # #     tokenized = tokenizer(batch['transcript'], truncation=True, padding='max_length')

# # #     # Process audio into mel-spectrogram features
# # #     mel_transform = torchaudio.transforms.MelSpectrogram()
# # #     mel_features = [
# # #         mel_transform(torch.tensor(audio['array'])) for audio in batch['audio']
# # #     ]

# # #     # Map accents to numeric labels if necessary
# # #     accent_map = {'german': 0}  # Add more accents as needed
# # #     accents = [accent_map.get('german', -1) for _ in batch['transcript']]  # Use the same label for all

# # #     # Add fields to tokenized dictionary
# # #     tokenized['mel_features'] = mel_features
# # #     tokenized['accent'] = accents  # Explicitly add the accent field
# # #     tokenized['speaker_id'] = batch['speaker_id']  # Keep speaker_id as-is

# # #     return tokenized


def preprocess_data(batch):
    # Tokenize the transcript
    tokenized = tokenizer(batch['transcript'], truncation=True, padding='max_length')

    # Add a hardcoded accent value if missing
    accent_map = {'german': 0, 'english': 1} # Example mapping for accents
    if 'accent' not in batch:
        batch['accent'] = ['german'] * len(batch['transcript'])  # Default to 'german'

    # Map accents to numeric labels
    labels = [accent_map.get(accent, -1) for accent in batch['accent']]
    tokenized['labels'] = torch.tensor(labels, dtype=torch.float)

    return tokenized



In [76]:
# from transformers import AutoTokenizer

# # Initialize the tokenizer outside the function to avoid repeated initializations
# tokenizer = AutoTokenizer.from_pretrained('bert-base-german-dbmdz-uncased')

# def preprocess_data(batch):
#     # Tokenize the text
#     tokenized_batch = tokenizer(batch['transcript'], padding=True, truncation=True, max_length=512, return_tensors="pt")

#     # Handle accents if they are part of the dataset
#     if 'accent' in batch:
#         accent_map = {'german': 0, 'english': 1}  # Define your accent map as needed
#         accent_labels = [accent_map.get(accent, -1) for accent in batch['accent']]
#         tokenized_batch['accent_labels'] = accent_labels

#     return tokenized_batch



In [77]:
# Load a dataset in streaming mode
streaming_dataset = load_dataset("facebook/multilingual_librispeech", "german", split="train", streaming=True)

# Apply preprocessing to the dataset
tokenized_dataset = streaming_dataset.take(500).map(preprocess_data, batched=True)


Resolving data files:   0%|          | 0/48 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/63 [00:00<?, ?it/s]

In [78]:
# import pandas as pd
# from datasets import Dataset

# data_samples = []
# for i, sample in enumerate(dataset):
#     data_samples.append(sample)
#     if i == 20:  # Collect only the first 21 samples
#         break

# # Create a DataFrame
# df = pd.DataFrame(data_samples)
# df["accent"] = "german"

# hf_dataset = Dataset.from_pandas(df)
# # Print the Data

# df.columns

In [79]:
# !apt-get install tree

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 47.9 kB of archives.
After this operation, 116 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tree amd64 2.0.2-1 [47.9 kB]
Fetched 47.9 kB in 1s (88.5 kB/s)
Selecting previously unselected package tree.
(Reading database ... 124173 files and directories currently installed.)
Preparing to unpack .../tree_2.0.2-1_amd64.deb ...
Unpacking tree (2.0.2-1) ...
Setting up tree (2.0.2-1) ...
Processing triggers for man-db (2.10.2-1) ...


In [84]:
# Take 500 samples and preprocess
data_samples = list(streaming_dataset.take(10))
data_dict = {key: [d[key] for d in data_samples] for key in data_samples[0].keys()}
dataset = Dataset.from_dict(data_dict)

AttributeError: module 'numba' has no attribute 'core'

In [ ]:
# tokenized_dataset = dataset.map(
#     preprocess_data,
#     batched=True,
#     remove_columns=['original_path', 'begin_time', 'end_time', 'chapter_id', 'file', 'id']
# )


# Now use this function with dataset.map
tokenized_dataset = dataset.map(
    preprocess_data,
    batched=True,
    remove_columns=['original_path', 'begin_time', 'end_time', 'chapter_id', 'file', 'id']
)



In [ ]:
# @title
!tree

In [ ]:
# Split into train, validation, and test sets
split_dataset = tokenized_dataset.train_test_split(test_size=0.3, seed=42)
validation_test_split = split_dataset['test'].train_test_split(test_size=0.5, seed=42)

# Organize splits into a DatasetDict
final_splits = DatasetDict({
    'train': split_dataset['train'],
    'validation': validation_test_split['train'],
    'test': validation_test_split['test']
})


In [ ]:
from transformers import TrainingArguments
import wandb

wandb.login()

training_args = TrainingArguments(
    output_dir='./results',
    run_name='my_specific_run',  # Set a unique run name
    num_train_epochs=5,
    logging_steps=10,
    evaluation_strategy='steps',
    eval_steps=50,
    per_device_train_batch_size=4,  # Reduced batch size
    per_device_eval_batch_size=8,   # Reduced eval batch size
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    learning_rate=1e-5,
    save_steps=500,  # Save checkpoints less frequently
    save_strategy='steps',
    load_best_model_at_end=True,
    remove_unused_columns=True,  # Ensure unused columns are dropped
    report_to="wandb",  # This will ensure that the training progress is reported to wandb
    fp16=True,  # Enable mixed precision training
    gradient_accumulation_steps=8,  # Accumulate gradients to simulate a larger batch size
)

# You can now pass these training arguments to the Trainer


In [ ]:
# # Load the model
# from transformers import AutoModel


# model = AutoModel.from_pretrained('bert-base-multilingual-cased')
# assert isinstance(model, torch.nn.Module), "Model not loaded as a PyTorch module"

from transformers import AutoModelForSequenceClassification

# Replace AutoModel with a classification model
model = AutoModelForSequenceClassification.from_pretrained(
    'bert-base-multilingual-cased',
    num_labels=1  # Adjust for the number of labels (e.g., 1 for regression, or >1 for classification)
)



In [ ]:
# Print a sample from the tokenized dataset
print(final_splits['train'][0].keys())  # Verify that input_ids, attention_mask, and labels are present


In [ ]:
# Take the first example from the tokenized dataset
for example in tokenized_dataset.take(1):
    print(example)


In [ ]:
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=final_splits['train'],
    eval_dataset=final_splits['validation'],
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
)



In [ ]:
# Start training and evaluate on the test set
trainer.train()
evaluation_results = trainer.evaluate(final_splits['test'])
print(evaluation_results)


### Let's test it

In [ ]:
# import os
# print(os.listdir('./results/checkpoint-132'))


In [ ]:
!ls ./results

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load the model from the checkpoint
checkpoint_path = './results/checkpoint-5'
model = AutoModelForSequenceClassification.from_pretrained(checkpoint_path)

# Load the tokenizer from the original pretrained model
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')

print("Model and tokenizer loaded successfully.")


In [ ]:
german_text = "Guten Morgen! Wie geht es Ihnen?"  # Example German text
inputs = tokenizer(german_text, return_tensors="pt", truncation=True, padding='max_length')


In [ ]:
# def inference_accent(text, accent, noise, diffusion_steps=5, embedding_scale=1):
#     # Preprocess the input text and accent
#     inputs = preprocess_data(text, accent, tokenizer)

#     # Move inputs to the correct device
#     input_ids = inputs['input_ids'].to(device)
#     attention_mask = inputs['attention_mask'].to(device)
#     accent_tensor = inputs['accent'].to(device)

#     # Perform inference
#     with torch.no_grad():
#         outputs = model(
#             input_ids=input_ids,
#             attention_mask=attention_mask,
#             accent=accent_tensor,
#             noise=noise,
#             diffusion_steps=diffusion_steps,
#         )

#         # Example: Extract mel-spectrogram or similar output
#         wav = vocoder(outputs.mel_features)  # Replace with the actual vocoder used

#     return wav


In [ ]:
def inference_accent2(text, noise, diffusion_steps=5, embedding_scale=1, accent=None):
    # Process the text
    text = text.strip()
    text = text.replace('"', '')

    # Apply accent-specific transformations before phonemization
    if accent:
        text = apply_accent_adjustments(text, accent)

    # Phonemize the text (phonemizer does not depend on accent, so pass the transformed text)
    ps = global_phonemizer.phonemize([text])
    ps = word_tokenize(ps[0])
    ps = ' '.join(ps)

    # Tokenization
    tokens = textclenaer(ps)
    tokens.insert(0, 0)
    tokens = torch.LongTensor(tokens).to(device).unsqueeze(0)

    # Adjust embedding scale based on accent
    if accent:
        accent_map = {
            'german': 0.8,  # Example scale factor for German accent
            'english': 1.2,  # Example scale factor for English accent
            # Add other accents and their respective adjustments here
        }
        embedding_scale *= accent_map.get(accent, 1)

    # Perform the inference
    with torch.no_grad():
        input_lengths = torch.LongTensor([tokens.shape[-1]]).to(tokens.device)
        text_mask = length_to_mask(input_lengths).to(tokens.device)

        t_en = model.text_encoder(tokens, input_lengths, text_mask)
        bert_dur = model.bert(tokens, attention_mask=(~text_mask).int())
        d_en = model.bert_encoder(bert_dur).transpose(-1, -2)

        s_pred = sampler(noise,
                          embedding=bert_dur[0].unsqueeze(0), num_steps=diffusion_steps,
                          embedding_scale=embedding_scale).squeeze(0)

        s = s_pred[:, 128:]
        ref = s_pred[:, :128]

        d = model.predictor.text_encoder(d_en, s, input_lengths, text_mask)

        x, _ = model.predictor.lstm(d)
        duration = model.predictor.duration_proj(x)
        duration = torch.sigmoid(duration).sum(axis=-1)
        pred_dur = torch.round(duration.squeeze()).clamp(min=1)

        pred_dur[-1] += 5

        pred_aln_trg = torch.zeros(input_lengths, int(pred_dur.sum().data))
        c_frame = 0
        for i in range(pred_aln_trg.size(0)):
            pred_aln_trg[i, c_frame:c_frame + int(pred_dur[i].data)] = 1
            c_frame += int(pred_dur[i].data)

        # encode prosody
        en = (d.transpose(-1, -2) @ pred_aln_trg.unsqueeze(0).to(device))
        F0_pred, N_pred = model.predictor.F0Ntrain(en, s)
        out = model.decoder((t_en @ pred_aln_trg.unsqueeze(0).to(device)),
                            F0_pred, N_pred, ref.squeeze().unsqueeze(0))

    return out.squeeze().cpu().numpy()

def apply_accent_adjustments(text, accent):
    """
    Apply accent-specific changes to the text before phonemization.
    Example: Replace phonetic patterns, adjust pronunciation etc.
    """
    if accent == "german":
        text = text.replace('th', 't')  # German pronunciation of 'th'
    elif accent == "english":
        text = text.replace('r', 'ʁ')  # English pronunciation of 'r'
    return text


In [ ]:
# Pass the inputs to the model
with torch.no_grad():  # Disable gradients for inference
    outputs = model(**inputs)

# Get logits (predictions before applying softmax)
logits = outputs.logits

# Apply softmax to get probabilities
probabilities = torch.nn.functional.softmax(logits, dim=-1)

# Get the predicted class
predicted_class = torch.argmax(probabilities, dim=-1).item()

print(f"Predicted class: {predicted_class}")
print(f"Probabilities: {probabilities}")


In [ ]:
import torch

# Check if GPU is available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

In [ ]:
_accent = "german"

In [ ]:
help(global_phonemizer.phonemize)

In [ ]:
text = "Guten Morgen! Wie geht es Ihnen?"  # German text
noise = torch.randn(1, 1, 256).to(device)  # Generate noise

# Use the imported function
result = inference_accent2(text, noise, diffusion_steps=5, embedding_scale=1, accent="german")

# Play the audio
import IPython.display as ipd
display(ipd.Audio(result, rate=24000))
